# Hackaton Peren



To do :
* try classificication on std_fee?

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="white")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets, decomposition

from time import time
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

import plotly.express as px

# !pip install fuzzy-c-means
from fcmeans import FCM

In [2]:
df = pd.read_csv("enriched_dataset.csv", sep = ",")
df["std_fee"] = df.groupby(["restaurant.id"])["fee"].transform("std")
df = df.join(pd.get_dummies(df["day_of_week"], prefix="is"))
df

,observation_uuid,start_lat,start_lon,start_code_postal,end_lat,end_lon,end_code_postal,rain,heat,day_of_week,...,timestamp,haversine,std_fee,is_Friday,is_Monday,is_Saturday,is_Sunday,is_Thursday,is_Tuesday,is_Wednesday
0,0d742256-0b36-f3cd-0acf-eba4441030ae,48.8406,2.3220,75014,48.8262,2.3192,75014,2,5,Saturday,...,104,1.614272,0.349489,0,0,1,0,0,0,0
1,743c7e9d-2fde-b035-2452-bc39dbf2eed1,48.8472,2.3527,75005,48.8337,2.3880,75012,9,3,Tuesday,...,134,2.987862,0.169031,0,0,0,0,0,1,0
2,43893bc5-449c-56ef-3857-ebe679ffa869,48.9015,2.3704,75019,48.8814,2.3484,75009,1,9,Sunday,...,159,2.753596,0.381491,0,0,0,1,0,0,0
3,284884a2-55e5-c1d9-b036-4ad9ee5be9b9,48.8504,2.2902,75015,48.8716,2.2811,75016,6,2,Thursday,...,46,2.449528,0.332738,0,0,0,0,1,0,0
4,a0ce5b5f-ca56-3921-f942-2f1efed97f9e,48.8583,2.3894,75011,48.8623,2.3426,75001,2,6,Friday,...,98,3.452422,0.489898,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,a51e940c-d74a-4e60-a432-e35d461e3ae9,48.8333,2.3991,75012,48.8344,2.3875,75012,7,0,Monday,...,129,0.857811,0.396187,0,1,0,0,0,0,0
4796,53e82886-9411-723a-50b4-17eb7111f1d1,48.8359,2.3381,75014,48.8592,2.3036,75007,0,2,Tuesday,...,111,3.617391,0.549675,0,0,0,0,0,1,0
4797,648a1e4b-fccb-1162-5311-cb54a0665299,48.8469,2.2456,75016,48.8680,2.2988,75016,9,2,Thursday,...,23,4.544545,0.720986,0,0,0,0,1,0,0
4798,51d4e883-bac0-7cab-b734-5e6adf01d2ae,48.8409,2.3497,75005,48.8604,2.3856,75011,1,0,Thursday,...,20,3.406091,0.327054,0,0,0,0,1,0,0


## 1) Data enrichment

In [3]:
# Add normalized variables for clustering in the last classification step

def normalized_col(df, col_name, method="mean-std"):
    if method=="mean-std":
        return (df[col_name]-df[col_name].mean())/df[col_name].std()
    elif method=="min-max":
        return (df[col_name]-df[col_name].min())/(df[col_name].max()-df[col_name].min())


METHOD = "mean-std"

df["time.driving.normed"] = normalized_col(df, "time.driving", method=METHOD)
df["time.biking.normed"] = normalized_col(df, "time.biking", method=METHOD)
df["rain.normed"] = normalized_col(df, "rain", method=METHOD)
df["heat.normed"] = normalized_col(df, "heat", method=METHOD)
df["fee.normed"] = normalized_col(df, "fee", method=METHOD)
df["timestamp.normed"] = normalized_col(df, "timestamp", method=METHOD)
df["haversine.normed"] = normalized_col(df, "haversine", method=METHOD)

df["haversine.normed.squared"] = df["haversine.normed"]**2

In [4]:
print(len(df))
# df_out doit faire 4800 lignes

4800


## 2) Isolate restaurants with fixed fees

In [5]:
df_tmp = df.copy()

df_tmp["nb_diff_fees"] = df_tmp.groupby(["restaurant.id"])["fee"].transform("nunique")
restos_fixed_fee = df_tmp[df_tmp["nb_diff_fees"] == 1]["restaurant.id"].values

df2 = df[~df["restaurant.id"].isin(restos_fixed_fee)].copy()
df2

,observation_uuid,start_lat,start_lon,start_code_postal,end_lat,end_lon,end_code_postal,rain,heat,day_of_week,...,is_Tuesday,is_Wednesday,time.driving.normed,time.biking.normed,rain.normed,heat.normed,fee.normed,timestamp.normed,haversine.normed,haversine.normed.squared
0,0d742256-0b36-f3cd-0acf-eba4441030ae,48.8406,2.3220,75014,48.8262,2.3192,75014,2,5,Saturday,...,0,0,-0.582753,-1.023509,-0.859001,0.213983,-0.354267,0.281611,-0.846718,0.716932
1,743c7e9d-2fde-b035-2452-bc39dbf2eed1,48.8472,2.3527,75005,48.8337,2.3880,75012,9,3,Tuesday,...,1,0,-0.036656,0.306668,1.582417,-0.485213,-0.498022,0.859894,0.448507,0.201159
2,43893bc5-449c-56ef-3857-ebe679ffa869,48.9015,2.3704,75019,48.8814,2.3484,75009,1,9,Sunday,...,0,0,0.348401,0.245848,-1.207775,1.612374,0.364509,1.341796,0.227606,0.051805
3,284884a2-55e5-c1d9-b036-4ad9ee5be9b9,48.8504,2.2902,75015,48.8716,2.2811,75016,6,2,Thursday,...,0,0,0.268121,0.808939,0.536095,-0.834810,-0.210512,-0.836401,-0.059114,0.003494
4,a0ce5b5f-ca56-3921-f942-2f1efed97f9e,48.8583,2.3894,75011,48.8623,2.3426,75001,2,6,Friday,...,0,0,1.273786,1.107509,-0.859001,0.563581,0.220754,0.165955,0.886563,0.785994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,a51e940c-d74a-4e60-a432-e35d461e3ae9,48.8333,2.3991,75012,48.8344,2.3875,75012,7,0,Monday,...,0,0,-1.410553,-1.174248,0.884869,-1.534006,-0.929288,0.763513,-1.560022,2.433669
4796,53e82886-9411-723a-50b4-17eb7111f1d1,48.8359,2.3381,75014,48.8592,2.3036,75007,0,2,Tuesday,...,1,0,1.022370,1.084810,-1.556550,-0.834810,0.939531,0.416544,1.042121,1.086016
4797,648a1e4b-fccb-1162-5311-cb54a0665299,48.8469,2.2456,75016,48.8680,2.2988,75016,9,2,Thursday,...,0,0,0.587799,1.848403,1.582417,-0.834810,0.795775,-1.279751,1.916379,3.672510
4798,51d4e883-bac0-7cab-b734-5e6adf01d2ae,48.8409,2.3497,75005,48.8604,2.3856,75011,1,0,Thursday,...,0,0,0.461851,0.548782,-1.207775,-1.534006,0.220754,-1.337579,0.842875,0.710439


In [6]:
df_fixed_fee = df[df["restaurant.id"].isin(restos_fixed_fee)].copy()
df_fixed_fee["algorithm"] = 1

## 3) Isolate datapoint where price is explained by one or several variables (R² > threshold, mainly 90 or 95%)


In [7]:
def fct_R2_from_reg_lin_of_fee_on_x_per_resto(X_vars, df_for_reg, seuil=0.9):
    list_reg_scores = []
    for resto in df_for_reg["restaurant.id"].unique():
        # regression on the (only) 8 data point for a restaurant
        dff = df_for_reg[df_for_reg["restaurant.id"] == resto]
        if len(X_vars) == 1:
            X = np.array(dff[X_vars[0]]).reshape(-1, 1) 
        if len(X_vars) > 1:
            X = np.array([dff[x_var] for x_var in X_vars]).T
        y = np.array(dff["fee"]).reshape(-1, 1) 
        reg = LinearRegression().fit(X, y)
        reg_score = reg.score(X, y)
        list_reg_scores.append(reg_score)
    df_reg_scores = pd.DataFrame({"restaurant.id" : df_for_reg["restaurant.id"].unique(),
                                  "linear_reg_scores" : list_reg_scores})

    print("NB restaurants catched by reg lin:", len(df_reg_scores[df_reg_scores["linear_reg_scores"] > seuil]))
    return df_reg_scores

### a) trouver restos où prix = a + b dist
Note: The LinearRegression module has a fit_intercept parameter. If you set that to True it will fit the intercept and so will add a column of ones. If you set it to False it will not.

In [8]:
seuil = 0.93

df_linear_reg_scores = fct_R2_from_reg_lin_of_fee_on_x_per_resto(X_vars=["haversine.normed"],
                                                                 df_for_reg=df2, seuil=seuil)

list_expl_lin = df_linear_reg_scores[df_linear_reg_scores["linear_reg_scores"] > seuil]["restaurant.id"].values
df_expl_lin = df2[df2["restaurant.id"].isin(list_expl_lin)].copy()
df_expl_lin["algorithm"] = 2

df3 = df2[~ df2["restaurant.id"].isin(list_expl_lin)].copy()
print("Nb obs fixed fees:", len(df_fixed_fee))
print("Nb obs reg lin haversine:", len(df_expl_lin))
print("Nb obs remaining:", len(df3))
len_sum = len(df_fixed_fee) + len(df_expl_lin) + len(df3)
print("Sum of lenghts (must be 4800):", len_sum, "-> " + "NOT "*(len_sum!=4800) + "OK") # => ok si 4800

NB restaurants catched by reg lin: 100
Nb obs fixed fees: 240
Nb obs reg lin haversine: 800
Nb obs remaining: 3760
Sum of lenghts (must be 4800): 4800 -> OK


### b) trouver restos ou fee = a + b * distance + c * timestamp 

In [9]:
seuil = 0.92

df_lin_reg_scores_haversine_timestamp = fct_R2_from_reg_lin_of_fee_on_x_per_resto(X_vars=["haversine.normed", "timestamp.normed", "is_weekend"],
                                         df_for_reg=df3, seuil=seuil)

list_expl_haversine_timestamp = df_lin_reg_scores_haversine_timestamp[df_lin_reg_scores_haversine_timestamp["linear_reg_scores"] > seuil]["restaurant.id"].values
df_expl_haversine_timestamp = df3[df3["restaurant.id"].isin(list_expl_haversine_timestamp)].copy()
df_expl_haversine_timestamp["algorithm"] = 3

df4 = df3[~ df3["restaurant.id"].isin(list_expl_haversine_timestamp)].copy()


len_sum = len(df_fixed_fee) + len(df_expl_lin) + len(df_expl_haversine_timestamp) + len(df4)
print("Sum of lenghts (must be 4800):", len_sum, "-> " + "NOT "*(len_sum!=4800) + "OK") # => ok si 4800

NB restaurants catched by reg lin: 274
Sum of lenghts (must be 4800): 4800 -> OK


### c) trouver restos ou fee = a + b * distance ² 

In [10]:
[col for col in df.columns if 'is' in col][1:]

['is_Friday',
 'is_Monday',
 'is_Saturday',
 'is_Sunday',
 'is_Thursday',
 'is_Tuesday',
 'is_Wednesday']

In [30]:
seuil = 0.85

df_sq_lin_reg_scores = fct_R2_from_reg_lin_of_fee_on_x_per_resto(X_vars=["timestamp.normed", "haversine.normed.squared"], df_for_reg=df4, seuil=seuil)

list_expl_sq_x_x2 = df_sq_lin_reg_scores[df_sq_lin_reg_scores["linear_reg_scores"] > seuil]["restaurant.id"].values
df_expl_sq_x_x2 = df4[df4["restaurant.id"].isin(list_expl_sq_x_x2)].copy()
df_expl_sq_x_x2["algorithm"] = 4

df5 = df4[~ df4["restaurant.id"].isin(list_expl_sq_x_x2)].copy()


len_sum = len(df_fixed_fee) + len(df_expl_lin) + len(df_expl_haversine_timestamp) + len(df_expl_sq_x_x2) + len(df5)
print("Sum of lenghts (must be 4800):", len_sum, "-> " + "NOT "*(len_sum!=4800) + "OK") # => ok si 4800

NB restaurants catched by reg lin: 14
Sum of lenghts (must be 4800): 4800 -> OK


## 4) Finalize by adding last algorithm for the remaining data

In [28]:
df5["algorithm"] = 5

print("NB restaurants remaining:", len(df5["restaurant.id"].unique()))

NB restaurants remaining: 152


In [29]:
# avec reg lin et reg **2
df_out = pd.concat([df_fixed_fee, df_expl_lin, df_expl_haversine_timestamp, df_expl_sq_x_x2, df5]) \
        .reset_index().sort_values("index", ignore_index=True).drop("index", axis=1)
print("NB restaurants per algorithm:")
display((df_out.groupby("algorithm").size()/8))
print("Size of results:", len(df_out))
df_out[["observation_uuid", "algorithm"]].to_csv("out.csv", index=False)

NB restaurants per algorithm:


algorithm
1     30.0
2    100.0
3    274.0
4     44.0
5    152.0
dtype: float64

Size of results: 4800


# 2nd method

On va ajuter des scores de régressions directement dans df2

In [8]:
def get_resto_reg_scores(data, columns):
    resto_scores = {}
    for resto in data["restaurant.id"].unique():
        # regression on the (only) 8 data point for a restaurant
        df_resto = data[data["restaurant.id"] == resto]
        if len(columns) == 1:
            X = np.array(df_resto[columns[0]]).reshape(-1, 1) 
        if len(columns) > 1:
            X = np.array([df_resto[x_var] for x_var in columns]).T
        y = np.array(df_resto["fee"]).reshape(-1, 1)
        
        reg = LinearRegression().fit(X, y)
        resto_scores[resto] = reg.score(X, y)
    return resto_scores

In [9]:
reg_h = get_resto_reg_scores(df2, ["haversine.normed"])
reg_htw = get_resto_reg_scores(df2, ["haversine.normed", "timestamp.normed", "is_weekend"])
reg_d = get_resto_reg_scores(df2, [col for col in df.columns if 'is' in col][1:])
reg_h2 = get_resto_reg_scores(df2, ["haversine.normed.squared"])
reg_hh2 = get_resto_reg_scores(df2, ["haversine.normed", "haversine.normed.squared"])
reg_m = get_resto_reg_scores(df2, ["rain.normed", "heat.normed"])
reg_hm = get_resto_reg_scores(df2, ["haversine.normed", "rain.normed", "heat.normed"])

In [10]:
df2["reg.h"] = df2["restaurant.id"].apply(lambda resto: reg_h[resto])
df2["reg.htw"] = df2["restaurant.id"].apply(lambda resto: reg_htw[resto])
df2["reg.d"] = df2["restaurant.id"].apply(lambda resto: reg_d[resto])
df2["reg.h2"] = df2["restaurant.id"].apply(lambda resto: reg_h2[resto])
df2["reg.hh2"] = df2["restaurant.id"].apply(lambda resto: reg_hh2[resto])
df2["reg.m"] = df2["restaurant.id"].apply(lambda resto: reg_m[resto])
df2["reg.hm"] = df2["restaurant.id"].apply(lambda resto: reg_hm[resto])
df2

,observation_uuid,start_lat,start_lon,start_code_postal,end_lat,end_lon,end_code_postal,rain,heat,day_of_week,...,timestamp.normed,haversine.normed,haversine.normed.squared,reg.h,reg.htw,reg.d,reg.h2,reg.hh2,reg.m,reg.hm
0,0d742256-0b36-f3cd-0acf-eba4441030ae,48.8406,2.3220,75014,48.8262,2.3192,75014,2,5,Saturday,...,0.281611,-0.846718,0.716932,0.996213,0.996317,0.508772,0.034405,0.998681,0.207156,0.996789
1,743c7e9d-2fde-b035-2452-bc39dbf2eed1,48.8472,2.3527,75005,48.8337,2.3880,75012,9,3,Tuesday,...,0.859894,0.448507,0.201159,0.991635,0.995629,0.341667,0.766867,0.998022,0.080151,0.993003
2,43893bc5-449c-56ef-3857-ebe679ffa869,48.9015,2.3704,75019,48.8814,2.3484,75009,1,9,Sunday,...,1.341796,0.227606,0.051805,0.809348,0.988632,0.607362,0.139588,0.826423,0.018473,0.840589
3,284884a2-55e5-c1d9-b036-4ad9ee5be9b9,48.8504,2.2902,75015,48.8716,2.2811,75016,6,2,Thursday,...,-0.836401,-0.059114,0.003494,0.541478,0.815100,0.264516,0.351983,0.612352,0.323665,0.746612
4,a0ce5b5f-ca56-3921-f942-2f1efed97f9e,48.8583,2.3894,75011,48.8623,2.3426,75001,2,6,Friday,...,0.165955,0.886563,0.785994,0.876946,0.990860,0.115079,0.605695,0.877624,0.225385,0.897735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,a51e940c-d74a-4e60-a432-e35d461e3ae9,48.8333,2.3991,75012,48.8344,2.3875,75012,7,0,Monday,...,0.763513,-1.560022,2.433669,0.521240,0.664847,0.954494,0.131310,0.724206,0.314582,0.831556
4796,53e82886-9411-723a-50b4-17eb7111f1d1,48.8359,2.3381,75014,48.8592,2.3036,75007,0,2,Tuesday,...,0.416544,1.042121,1.086016,0.950699,0.999080,0.654846,0.091578,0.975073,0.002769,0.954479
4797,648a1e4b-fccb-1162-5311-cb54a0665299,48.8469,2.2456,75016,48.8680,2.2988,75016,9,2,Thursday,...,-1.279751,1.916379,3.672510,0.699587,0.907186,0.459521,0.438982,0.799668,0.104606,0.750859
4798,51d4e883-bac0-7cab-b734-5e6adf01d2ae,48.8409,2.3497,75005,48.8604,2.3856,75011,1,0,Thursday,...,-1.337579,0.842875,0.710439,0.014933,0.417703,0.345576,0.002937,0.023271,0.352597,0.879679


In [11]:
def choose_algo_max(data: pd.DataFrame, col_to_algo: dict):
    resto_algos = {}
    for _, row in data.iterrows():
        chosen_algo, current_score = 0, 0
        for col_name, algo in col_to_algo.items():
            if row.loc[col_name] > current_score:
                chosen_algo = algo
                current_score = row[col_name]
        resto_algos[row["restaurant.id"]] = (chosen_algo, current_score)
    return resto_algos

def choose_algo_threshold(data: pd.DataFrame, col_to_algo: dict):
    resto_algos = {}
    for _, row in data.iterrows():
        if row["reg.h"] > 0.94:
            col_name = "reg.h"
        elif row["reg.htw"] > 0.92:
            col_name = "reg.htw"
        elif row["reg.m"] > 0.7:
            col_name = "reg.m"
        elif row["reg.h2"] > 0.8:
            col_name = "reg.h2"
        else:
            col_name = "reg.d"
        chosen_algo = col_to_algo[col_name]
        current_score = row[col_name]
        resto_algos[row["restaurant.id"]] = (chosen_algo, current_score)
    return resto_algos

col_to_algos = {"reg.m": 2, "reg.htw": 3, "reg.d": 4, "reg.h2": 5, "reg.h": 6, "reg.hh2": 7}
df_resto_scores = df2.groupby("restaurant.id")[list(col_to_algos.keys())].mean().reset_index()

In [226]:
resto_algos_max = choose_algo_max(df_resto_scores, col_to_algos)
resto_algos_threshold = choose_algo_threshold(df_resto_scores, col_to_algos)

In [227]:
df2["algorithm"] = df2["restaurant.id"].apply(lambda x: resto_algos_threshold[x][0])
df2["reg.score"] = df2["restaurant.id"].apply(lambda x: resto_algos_threshold[x][1])

df2.groupby("algorithm").size()/8

df2

# avec reg lin et reg **2
concat_out = pd.concat([df_fixed_fee, df2]).reset_index().sort_values("index", ignore_index=True).drop("index", axis=1)
print("NB restaurants per algorithm:")
display((concat_out.groupby("algorithm").size()/8))
print("Size of results:", len(df_out))
concat_out[["observation_uuid", "algorithm"]].to_csv("out_2.csv", index=False)

NB restaurants per algorithm:


algorithm
1     30.0
2     22.0
3    286.0
4    165.0
5      9.0
6     88.0
dtype: float64

Size of results: 4800
